In [ ]:
# 初回のみ
%cd /app/notebook/
!git clone https://github.com/Vision-CAIR/MiniGPT-4.git

In [6]:
# 初回のみ
%cd /app/notebook/MiniGPT-4
!git checkout 22d8888 # latest version as of July 31, 2023.
!wget https://huggingface.co/rinna/bilingual-gpt-neox-4b-minigpt4/resolve/main/customized_mini_gpt4.py
!wget https://huggingface.co/rinna/bilingual-gpt-neox-4b-minigpt4/resolve/main/checkpoint.pth

/app/notebook/MiniGPT-4
HEAD is now at 22d8888 Update README.md
--2023-07-31 07:45:00--  https://huggingface.co/rinna/bilingual-gpt-neox-4b-minigpt4/resolve/main/customized_mini_gpt4.py
Resolving huggingface.co (huggingface.co)... 13.32.50.102, 13.32.50.129, 13.32.50.80, ...
Connecting to huggingface.co (huggingface.co)|13.32.50.102|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5964 (5.8K) [text/plain]
Saving to: ‘customized_mini_gpt4.py.1’

customized_mini_gpt 100%[===================>]   5.82K  --.-KB/s    in 0s      

2023-07-31 07:45:00 (3.90 GB/s) - ‘customized_mini_gpt4.py.1’ saved [5964/5964]

--2023-07-31 07:45:01--  https://huggingface.co/rinna/bilingual-gpt-neox-4b-minigpt4/resolve/main/checkpoint.pth
Resolving huggingface.co (huggingface.co)... 13.32.50.129, 13.32.50.53, 13.32.50.102, ...
Connecting to huggingface.co (huggingface.co)|13.32.50.129|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hugging

In [1]:
# 次回以降はここから
%cd /app/notebook/MiniGPT-4

/app/notebook/MiniGPT-4


In [80]:
import torch
import requests
from PIL import Image
from minigpt4.processors.blip_processors import Blip2ImageEvalProcessor
from customized_mini_gpt4 import CustomizedMiniGPT4

ckpt_path = "./checkpoint.pth"

model = CustomizedMiniGPT4(gpt_neox_model="rinna/bilingual-gpt-neox-4b")
tokenizer = model.gpt_neox_tokenizer

if torch.cuda.is_available():
    model = model.to("cuda")

if ckpt_path is not None:
    print("Load BLIP2-LLM Checkpoint: {}".format(ckpt_path))
    ckpt = torch.load(ckpt_path, map_location="cpu")
    model.load_state_dict(ckpt['model'], strict=False)

vis_processor = Blip2ImageEvalProcessor()

Loading VIT
freeze vision encoder
Loading VIT Done
Loading Q-Former
freeze Qformer
Loading Q-Former Done
Loading LLM
Loading LLM Done
Load BLIP2-LLM Checkpoint: ./checkpoint.pth


In [81]:
def inference(image_url):
    raw_image = Image.open(requests.get(image_url, stream=True).raw).convert('RGB')
    image = vis_processor(raw_image).unsqueeze(0).to(model.device)
    image_emb = model.encode_img(image)
    prompt = [
        {
            "speaker": "ユーザー",
            "text": "<Img><ImageHere></Img> 何が見えますか？"
        },
    ]
    prompt = [
        f"{uttr['speaker']}: {uttr['text']}"
        for uttr in prompt
    ]
    prompt = "\n".join(prompt)
    prompt = (
        prompt
        + "\n"
        + "システム: "
    )
    print(prompt)

    embs = model.get_context_emb(prompt, [image_emb])

    output_ids = model.gpt_neox_model.generate(
        inputs_embeds=embs,
        max_new_tokens=512,
        do_sample=True,
        temperature=1.0,
        top_p=0.85,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

    output = tokenizer.decode(output_ids.tolist()[0], skip_special_tokens=True)
    print(output)

In [86]:
inference("https://ogre.natalie.mu/media/news/comic/2023/0325/jyuyutsu2_PV1_06.jpg?impolicy=hq&imwidth=730&imdensity=1")

3人の男性が立っている


In [89]:
inference("https://upload.wikimedia.org/wikipedia/commons/1/15/Red_Apple.jpg")

りんご


In [94]:
inference("https://img.cpcdn.com/recipes/4174680/640x480c/13c9ffb6dfb8c32eacda7b8b4628208c.jpg?u=11688791&p=1478826339")

リンゴの断面


In [100]:
inference("https://collabo-cafe.com/wp-content/uploads/cadbb15fbe6e5c8c2878027820dfde30.jpg")

アニメキャラ、キャラクター、キャラ絵、アニメ、キャラ、漫画、キャラ絵


In [106]:
inference("https://static.mercdn.net/c!/w=240/thumb/photos/m55026016864_1.jpg?1689509667")

アニメの女の子


In [111]:
inference("https://blog.ikehikoshop.jp/wp-content/uploads/2021/09/coin.jpg")

洗濯機です。


In [125]:
inference("https://ogre.natalie.mu/media/news/comic/2022/1217/5_toji.jpg?imwidth=750&imdensity=1")

黒い布が巻かれた手
